In [ ]:
import numpy as np
import pandas as pd

np.random.seed(123)  # For reproducibility

# Generate synthetic multivariate dataset with outliers
n_samples = 1000  # Number of samples
n_features = 5  # Number of features
n_outliers = np.random.randint(100, 150)  # Number of outliers
mean = np.zeros(n_features)
cov = np.identity(n_features)
data = np.random.multivariate_normal(mean, cov, n_samples)
outliers = np.random.uniform(low=-10, high=10, size=(n_outliers, n_features))
data = np.concatenate((data, outliers), axis=0)
data = pd.DataFrame(data, columns=[f'Feature_{i+1}' for i in range(n_features)])
data.to_csv('synthetic_dataset.csv', index=False)
